# PC-Lab 9: Support Vector Machines

## Introduction

Support vector machines (SVMs) are (still) among the most popular techniques used for classification and regression. SVM is like a gifted artist, seeing the world not just for what it is, but for the potential spaces in between. It asks a fundamental question: "Given these points, where should the line (or plane, or hyperplane) be drawn to best separate them?" In mathematical terms, it tries to find a hyperplane that maximizes the margin between two classes. 

But the true genius of SVM reveals itself in non-linear situations. 
As these methods only use instances as products of the feature vectors, they can be kernelized by means of the kernel trick:

\begin{align}
k(\mathbf{x_i},\mathbf{x_j}) &= \phi(\mathbf{x_i})^T\phi(\mathbf{x_j}). 
\end{align}

SVMs are what people call a _maximum margin classifier_, as it can be shown that the mathematical framework of an SVM maximizes the distance between the decision boundary and the instances closest to the boundary. In addition, they are considered sparse methods because after the training phase a part of the data set can usually be discarded, retaining only the _support vectors_. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/SVM_margin.png/1920px-SVM_margin.png" width=500>

Technically, the SVM tries to maximize the margin $M$, while retaining an optimal classification error. As is shown in theory lecture, the first is equal to $||\mathbf{w}||^{-1}$ and so this is equivalent to minimizing: 
\begin{equation}
    \text{arg min} \frac{1}{2} ||\mathbf{w} + w_0||^2, 
\end{equation}

subject to: 
\begin{equation}
    y_i(\mathbf{x}^T(\mathbf{w} + w_0)) \geq 1, \qquad \forall i =  1, ..., n. 
\end{equation}

However, this implementation only works for data that is separable. By allowing misclassifications one can turn the margin classifier into the well-known SVM by introducing slack variables $\xi_i$: 
\begin{equation}
    \text{arg min}_{\mathbf{w} + w_0} \{ \frac{1}{2} ||\mathbf{w} + w_0||^2 + C \sum_{i=1}^n \xi_i \}, 
\end{equation}

subject to: 
\begin{equation}
    y_i(\mathbf{x}^T(\mathbf{w}+w_0)) \geq 1-\xi_i, \qquad \forall i =  1, ..., n. 
\end{equation}

The above formulation can be translated to an equivalent _dual representation_, which leads to the following maximization problem:
\begin{equation}
    \text{arg max}_{a_i} \{ \sum_{i=1}^n a_i - \frac{1}{2} \sum_{i=1}^n \sum_{k=1}^n a_i a_k y_i y_k \mathbf{x_i}^{T}\mathbf{x_k} \}, 
\end{equation}

subject to: 
\begin{align}
    0 \leq a_i &\leq C, \\
    \sum_{i=1}^n a_iy_i &= 0, \qquad \forall i = 1, ..., n, 
\end{align}

Due to the dot products in the above formulation, we can apply the **kernel trick**, which then gives us the final maximization problem:
\begin{equation}
    \text{arg max}_{a_i} \{ \sum_{i=1}^n a_i - \frac{1}{2} \sum_{i=1}^n \sum_{k=1}^n a_i a_k y_i y_k k(\mathbf{x_i},\mathbf{x_k}) \}, 
\end{equation}

subject to: 
\begin{align}
    0 \leq a_i &\leq C, \\
    \sum_{i=1}^n a_iy_i &= 0, \qquad \forall i = 1, ..., n, 
\end{align}

where $k$ is defined as the kernel function. 

In this PC-lab we will use SVMs for both classification and regression purposes. In the first part we will predict the label of two bacterial species  based on flow cytometry data. Next, we will try to predict the molecular function of a set of proteins by using only its amino acid sequence by using a customized spectral kernel. Finally, support vector regression (SVR) will be used to have a second attempt at the prediction of the strength of concrete. 

## Grid search vs. randomized grid search

<img src="https://miro.medium.com/v2/format:webp/1*ZTlQm_WRcrNqL-nLnx6GJA.png" width=600>

SVMs typically contain multiple hyperparameters. First of all there is the penalty of regularization (often noted as $C$ for SVMs, equivalent to $1/\lambda$ previously seen in theory). Second there is a kernel to choose, and depending on the kernel, there can be one (or sometimes more) hyperparameters to tune (often denoted with a $\gamma$). One can even include the type of kernel as a hyperparameter. It is straightforward to note that the computational time of training a model increases exponentially with the number of hyperparameters that needs to be  tuned. 

Two searches are often proposed to tune a machine learning model in function of its hyperparameters: 
- the first considers an exhaustive and structured search in the full hyperparameter space; 
- the second applies random combinations of hyperparameter settings. 

[It has been shown that when only few hyperparameters have an impact on model performance a randomized grid search is preferred. ](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) 

`Scikit-learn` offers implementations of [both](http://scikit-learn.org/stable/modules/grid_search.html), of which we will compare the use in this PC-lab. 

## Support vector classification of microbial flow cytometry data

#### Dataset
We will have a second look at the flow cytometry dataset. The 'SC' features measure scatter, and say something about the morphologhy of the cells (FSC: forward scatter, SSC: sideway scatter). The 'FL' features are fluorescence features from different parts of the spectrum. Two bacterial species have been measured separately, _Shewanella oneidensis_ and _Sphingomonas aromaticivorans_, labelled 1 and 0 respectively. Scikit-learn provides [various implementations of SVMs](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm), which we will use to analyze this dataset. 

<div class="alert alert-success">
<h1>EXERCISE 9.1</h2>
<p><b>
a) Read in the data from the files `fc_train` and `fc_test`. Often, before flow cytometry data can be analyzed, mathematical transformations are applied to for example visualize the data. Therefore we first need to preprocess the data by performing an $\text{arcsinh}(x)$ transformation and normalization (or standardization). Why do we need to normalize for SVMs? Sample 10% of the train data (due to long runtimes) and perform the suggested preprocessing steps. 

b) Fit and optimize a SVM with a linear kernel to the training data. The 'species' label denotes the target to predict, you can ignore the 'Width' and 'Time' feature. Evaluate the performance on the test set, for different values of the regularization parameter. What's the 'optimal' performance of this model? Which hyperparameter setting goes along with this? What does this mean? 

c) Explore other kernels to tackle this dataset. Find optimal hyperparameters using a grid search. 

d) Another possibility is to use a randomized grid search. How long did previous exercise take? How long does a randomized grid search take before you reach the same performance? What do you prefer? 

e) What would be an alternative option to tune your model? 
</b>
</div>

In [55]:
import numpy as np
import urllib.request

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import SVR

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from IPython.display import display, HTML

plt.style.use('seaborn-v0_8')

Download the data and read them into dataframes:

In [6]:
# load data
urllib.request.urlretrieve('https://raw.githubusercontent.com/BioML-UGent/MLLS/main/08_svms/fc_train.csv', 'fc_train.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/BioML-UGent/MLLS/main/08_svms/fc_test.csv', 'fc_test.csv')  

# read data into pandas dataframe
df_train = pd.read_csv('fc_train.csv', header=0, index_col=0)
df_test = pd.read_csv('fc_test.csv', header=0, index_col=0)


##### 1a) Sample a fraction from the train data, perform the transformation, scale the data:

In [ ]:

#Tip: np.arcsinh()
# sample 10 % of the train data (with replacement) for training: note: use 'species' as weights
fc_train = ".." # TODO
# take all but 'Width', 'Time' and 'species' column names for features
features = ".." # TODO
# take 'species' column as target

# transform data with arcsinh
fc_train_trans = ".." # TODO
fc_test_trans = ".." # TODO

# scale data using StandardScaler
".." # TODO
# fit to train data and transform train and test data
".." # TODO

# convert to pandas dataframe
".." # TODO

##### 1b) Fit and optimize an SVM

In [ ]:
##1b): 
acc = np.zeros(10)
i=0
# iterate over different values for C
for c in np.logspace(-4,4,10): 

    ".." # TODO

    
    acc[i] = ".."   # TODO
    i+=1
    
fig, ax = plt.subplots()
ax.scatter(np.logspace(-4,4,10), acc, c='r', marker='x', s=50)
ax.set_xlabel('C', size=15)
ax.set_ylabel('Accuracy', size=15)
ax.set_xscale('log')
ax.set_xlim(10**(-5),10**5)
ax.set_ylim(0.75,0.95)

plt.title('Linear SVM for different values of C', size=15)
plt.show()

print('Performance Linear SVM: ' + str(acc.max()))
print("Optimal C: " + str(np.logspace(-4,4,10)[acc.argmax()]))

##### 1c) perform GridSearch over c using other kernels
Use the scikit-learn class [both](http://scikit-learn.org/stable/modules/grid_search.html), optimize over the regularization strength as well as the kernel parameters

In [ ]:
##1c): 
svc_gs = ".." # TODO
".." # TODO
y_pred = ".." # TODO    
acc_gs = ".." # TODO
print('Accuracy SVM with rbf-kernel after grid search: ' + str(acc_gs))

In [ ]:
svc_gs.best_params_
print(f"Best parameters: {svc_gs.best_params_} with an accurcy of {acc_gs}")

#### 1d) compare with Random Grid Search:

In [ ]:
#1d):
".." # TODO

In [ ]:
print(f'Best parameters: {svc_rs.best_params_} with an accurcy of {acc_rs}')

## Support Vector Machines with custom kernels

The only requirement for a valid kernel is that it is positive definite and symmetric. Therefore, it is possible to define a **kernel function over complex structures**, making kernel methods particularly useful in bioinformatics,
where the instances are often sequences, graphs, trees, molecules or texts, rather than vectors of fixed length. However, the use of these types of kernels can be computationally demanding. 

In this last part of the PC-lab we will use a _spectrum kernel_ for predicting the molecular function of a set of protein sequences. This kernel is based on the number of $k$-mers that two sequences have in common. Suppose that each sequence is denoted as $\mathbf{x_i} \in \Sigma^∗$ where $\Sigma$ is the alphabet of amino acids or nucleotides, depending on the application. A $k$-mer $\mathbf{a} \in \Sigma_k$ is a sequence of length $k$. The sequence $\mathbf{x_i}$ contains $\mathbf{a}$ iff $\mathbf{x_i} = \mathbf{uav}$ with $\boldsymbol{u}$, $\boldsymbol{v}$ being some other (possibly of lengtgh $0$) sequences.

Let $N(\mathbf{a}, \mathbf{x_i})$ be the number of times $\mathbf{a}$ appears in sequence $\mathbf{x_i}$. With this notation in mind, the _spectrum kernel function_ between two strings $\mathbf{x_i}$ and $\mathbf{x_j}$ is defined as:
\begin{equation}
    SK_k(\mathbf{x_i},\mathbf{x_j}) = \sum_{\mathbf{a} \in \Sigma_k} N(\mathbf{a},\mathbf{x_i}) \times N(\mathbf{a},\mathbf{x_j}). 
\end{equation}

This kernel is biased toward sequences that contain multiple instances of the same k-mer. This can easily be seen when considering a pair of sequences that contains the same $k$-mer twice. Such a pair gains a score of 4, while having two different $k$-mers in common only gives a score of 2. For this reason a normalized spectrum kernel can be used as alternative, defined as:
\begin{equation}
    NSK_k(\mathbf{x_i},\mathbf{x_j}) = \sum_{\mathbf{a} \in \Sigma_k} \text{min}\left( N(\mathbf{a},\mathbf{x_i}),N(\mathbf{a},\mathbf{x_j})\right). 
\end{equation}

The normalized kernel has an explicitly defined mapping $\phi(\mathbf{x})$. If the maximum length of the sequences is $n$, then $\phi(\mathbf{x})$ defines a map of $\mathbf{x} \in \Sigma^n$ to a feature space of dimension $|\Sigma^kn|$, where each dimension is indexed by a $k$-mer a and an integer $1 ≥ i ≥ n$. The mapping is as follows:
\begin{equation}
\phi_{(\mathbf{a},i)}(\mathbf{x}) = \left\{
    \begin{array}{ll}
        1 & \text{if $k$-mer $\mathbf{a}$ appears at least $i$ times in $\mathbf{x}$}, \\
        0 & \text{otherwise}. 
    \end{array}
\right.
\end{equation}

Using these two kernels, we will analyze 150 protein sequences and predict their functional label. Each molecule belongs either to the category `transport`, `glycosylation` or `signaling`. 

`Calculate (Normalized) Spectrum Kernel`: 

**Input**: Sequence 1, Sequence 2, $k$; 

**1. ** Find all possible substrings of length $k$ in sequence 1. 

**2. ** Find all possible substrings of length $k$ in sequence 2.

**3. ** Find common subset of substrings present in sequence 1 and sequence 2. 

**4. ** Count the number of times a specific substring is present in sequence 1 and 2 and calculate $SK$(sequence1,sequence2) or $NSK$(sequence1,sequence2) according to the formulas above. 

<div class="alert alert-success">
<h1> EXERCISE 9.2 </h2>
<p><b>
In attachment to this PC-lab, you can find two files called `Spectrum_Kernel.csv` and `Spectrum_Kernel_norm.csv`. The labels of the proteins can be found in the file `ProteinFunctionalLabels.txt`. These files contain the gram matrices generated according to the code above. 

a) What kind of classification problem is this? How can this problem be approached? 

b) Which kernel representation works better? Evaluate the spectrum kernels using a SVM. Use 70% of the data for training and 30% for testing. Think about the properties of your kernel matrices. How should those training and test sets be constructed? How should the kernel matrix be fed to the SVM? Don't forget to tune the complexity parameter $C$. 
</b>
</div>

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/BioML-UGent/MLLS/main/08_svms/Spectrum_Kernel.csv', 'Spectrum_Kernel.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/BioML-UGent/MLLS/main/08_svms/Spectrum_Kernel_norm.csv', 'Spectrum_Kernel_norm.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/BioML-UGent/MLLS/main/08_svms/ProteinFunctionalLabels.txt', 'ProteinFunctionalLabels.txt')


#Some preprocessing

spectrum_kernel = pd.read_csv('Spectrum_Kernel.csv', header=None, index_col=None)
spectrum_kernel_norm = pd.read_csv('Spectrum_Kernel_norm.csv', header=None, index_col=None)
labels = pd.read_table('ProteinFunctionalLabels.txt',header=None, index_col=None)
 
y = LabelEncoder().fit_transform(labels)
y = pd.DataFrame(y,dtype=int)

#### 2b) construct train and test sets:

In [ ]:
# split data into train and test set
X_train, X_test, y_train, y_test = ".." # TODO
X_train_norm, X_test_norm, y_train_norm, y_test_norm = ".." # TODO

# important: index both train and test dataframes with the same (train) indices
X_train = ".." # TODO
X_test = ".." # TODO
X_train_norm = ".." # TODO
X_test_norm = ".." # TODO

#### 2 b) Tackle the classification problem using SVM. 
Tip: set kernel to 'precomputed' in the SVC class.
Again try different values for $C$.

In [ ]:
acc = np.zeros(50)
i=0
for c in np.logspace(-4,-1,50): 
    svc = ".." # TODO
    
    ".." # TODO

    acc[i] = ".." # TODO
    i+=1
        
fig, ax = plt.subplots()
ax.scatter(np.logspace(-4,-1,50), acc, c='r', marker='x', s=50)
ax.set_xlabel('C', size=18)
ax.set_ylabel('Accuracy', size=15)
ax.set_xscale('log')
ax.set_xlim(10**(-5),10**0)
ax.set_ylim(0.1,1)
plt.title('Spectrum Kernel SVM for different values of C', size=15)
plt.show()

print('Protein label identification performance using Spectrum Kernel: ' + str(acc.max()))
print("Optimal C: " + str(np.logspace(-4,-1,50)[acc.argmax()]))

Optional: visualize confusion matrix using the found optimal parameters.

In [ ]:
".." # TODO

Now compare with the results you get when you used the normalized Gram matrix:

In [ ]:
".." # TODO



Optional: visualize the confusion matrix.

In [ ]:
".."    # TODO

## Support vector regression

SVMs can also be adapted to be used for regression problems, using a so-called $\epsilon$-_insensitive error function_, which returns a zero error if the absolute difference between the prediction and target is less than $\epsilon$, with $\epsilon > 0$. If we note predictions for instance $i$ as $y_i^*$, we can write the error functions as follows: 
\begin{align}
E_\epsilon(y_i - y_i^*) &= 0, \quad \text{if} \quad |y_i - y_i^*| < \epsilon, \\
E_\epsilon(y_i - y_i^*) &=  |y_i - y_i^*| - \epsilon, \quad \text{otherwise}. 
\end{align}

As we now need two kinds of slack variables (why is that?), the mathematical optimization scheme now looks as follows: 

\begin{equation}
    \text{arg min}_{a_i,a_i^*} \{ \epsilon\sum_{i=1}^n (a_i+a_i^*) -  \frac{1}{2}\sum_{i=1}^n \sum_{k=1}^n (a_i-a_i^*) (a_k-a_k^*) K + \sum_{i=1}^n (a_i - a_i^*)y_i \} , 
\end{equation}

subject to: 
\begin{align}
    0 &\leq a_i, \\
    a_i^* &\leq C, \\
    \sum_{i=1}^n (a_i - a_i^*) &= 0, \\
    a_ia_i^* &= 0, \qquad \forall i = 1, ..., n.   
\end{align}

Alternative formulations of SVMs are also possible. One of them are called $\nu$-SVMs, where the amount of support vectors $\nu$ are controlled. Those interested can have a look [here](http://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVR.html#sklearn.svm.NuSVR). 

<div class="alert alert-success">
<h1>EXERCISE 9.3 (Optional)</h2>
<p><b>
We will have a second look at the `Concrete Compressive Strength` dataset. We are going to apply a [support vector regression model](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) to the dataset. 

a) Which hyperparameters do you have in this setting? How would you optimize the model? 

b) Fit a support vector regression model to the data. Start with the rbf kernel. Evaluate your performance on a 30% held-out test set in terms of the mean squared error (MSE). Can you do better than kernel ridge regression? If the training takes too long, try to use a subsample of the data. 
</b>
</div>

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/BioML-UGent/MLLS/main/07_kernels/concreteComprStrength.txt", "concreteComprStrength.txt")

df = pd.read_table('concreteComprStrength.txt', delim_whitespace=True, header=0, index_col=None)
df = df.sample(frac=1)
features = ['cement', 'blastFurnaceSlag', 'flyAsh', 'water', 'superelastizer', 'coarseAggregate', 'fineAggregate', 'age']
target = ['compressiveStrength']

X_train, X_test, y_train, y_test = train_test_split(df.loc[:,features], df.loc[:,target], test_size=0.30, random_state=85)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = pd.DataFrame(scaler.transform(X_train), columns=features)
X_test_stand = pd.DataFrame(scaler.transform(X_test), columns=features)


In [ ]:
".." # TODO